In [14]:
!apt-get update
!pip install pyspark findspark

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [15]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

# 5. Extraction des données

In [16]:
import requests

In [17]:
url = "https://data.lillemetropole.fr/data/ogcapi/collections/ilevia:vlille_temps_reel/items?f=json&limit=-1"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("données Vlille DL")
else:
    print(f"erreur: {response.status_code}")


données Vlille DL


In [18]:
print(data)
print(type(data))

{'numberMatched': 302, 'numberReturned': 302, 'records': [{'@id': 'vlille_temps_reel.114', 'nom': 'BOIS HABITE', 'adresse': 'Face au 533 avenue du Président Hoover', 'code_insee': None, 'commune': 'Lille', 'etat': 'EN SERVICE', 'type': 'AVEC TPE', 'nb_places_dispo': 2, 'nb_velos_dispo': 15, 'etat_connexion': 'CONNECTÉ', 'x': 3.079211, 'y': 50.62651, 'date_modification': '2025-03-24T09:53:24.977+00:00'}, {'@id': 'vlille_temps_reel.119', 'nom': 'LOMME-LAMBERSART', 'adresse': '388 Avenue de Dunkerque', 'code_insee': None, 'commune': 'Lambersart', 'etat': 'EN SERVICE', 'type': 'AVEC TPE', 'nb_places_dispo': 6, 'nb_velos_dispo': 12, 'etat_connexion': 'CONNECTÉ', 'x': 3.018803, 'y': 50.640957, 'date_modification': '2025-03-24T09:53:24.978+00:00'}, {'@id': 'vlille_temps_reel.1', 'nom': 'METROPOLE EUROPEENNE DE LILLE', 'adresse': 'MEL RUE DU BALLON', 'code_insee': None, 'commune': 'LILLE', 'etat': 'RÉFORMÉ', 'type': 'AVEC TPE', 'nb_places_dispo': 0, 'nb_velos_dispo': 0, 'etat_connexion': 'DÉCO

In [19]:
records = data.get("records", [])
print(type(records))
print(records[:1])

<class 'list'>
[{'@id': 'vlille_temps_reel.114', 'nom': 'BOIS HABITE', 'adresse': 'Face au 533 avenue du Président Hoover', 'code_insee': None, 'commune': 'Lille', 'etat': 'EN SERVICE', 'type': 'AVEC TPE', 'nb_places_dispo': 2, 'nb_velos_dispo': 15, 'etat_connexion': 'CONNECTÉ', 'x': 3.079211, 'y': 50.62651, 'date_modification': '2025-03-24T09:53:24.977+00:00'}]


In [20]:
extracted_data = [{
        'id': record.get('@id', None),
        'nom': record.get('nom', None),
        'etat' : record.get('etat', None),
        'nb_places_dispo' : record.get('nb_places_dispo', None),
        'nb_velos_dispo' : record.get('nb_velos_dispo', None),
        'etat_connexion' : record.get('etat_connexion', None),
        'x' : record.get('x', None),
        'y' : record.get('y', None),
        'date' : record.get('date_modification', None),
    } for record in records]


df_spark = spark.createDataFrame(extracted_data)
df_spark.show(5)

+--------------------+----------+--------------+--------------------+---------------+--------------+--------------------+--------+---------+
|                date|      etat|etat_connexion|                  id|nb_places_dispo|nb_velos_dispo|                 nom|       x|        y|
+--------------------+----------+--------------+--------------------+---------------+--------------+--------------------+--------+---------+
|2025-03-24T09:53:...|EN SERVICE|      CONNECTÉ|vlille_temps_reel...|              2|            15|         BOIS HABITE|3.079211| 50.62651|
|2025-03-24T09:53:...|EN SERVICE|      CONNECTÉ|vlille_temps_reel...|              6|            12|    LOMME-LAMBERSART|3.018803|50.640957|
|2022-11-29T10:47:...|   RÉFORMÉ|    DÉCONNECTÉ| vlille_temps_reel.1|              0|             0|METROPOLE EUROPEE...|3.075992|50.641926|
|2022-11-29T10:47:...|   RÉFORMÉ|    DÉCONNECTÉ|vlille_temps_reel...|              0|             0|              VERDUN| 3.10231|50.638126|
|2022-11-29T1

In [21]:
# df_spark.select("id").show(truncate=False)